In [ ]:
!pip install langchainhub langchain-community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.0 MB/s eta 0:00:00


In [ ]:
kb_facts = [
    {"animal": "dog", "type": "mammal"},
    {"animal": "whale", "type": "mammal"},
    {"animal": "eagle", "type": "bird"},
    {"animal": "shark", "type": "fish"},
    {"animal": "frog", "type": "amphibian"},
    {"animal": "bat", "type": "mammal"},
    {"animal": "penguin", "type": "bird"},
    {"animal": "crocodile", "type": "reptile"},
    {"animal": "cat", "type": "mammal"},
    {"animal": "sparrow", "type": "bird"},
]

kb_rules = [
    "All mammals are warm-blooded.",
    "Birds have feathers.",
    "Reptiles lay eggs.",
    "Fish live in water.",
    "Amphibians live both on land and in water.",
    "Warm-blooded animals maintain constant body temperature."
]


In [ ]:
# Imports
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.documents import Document
from langchain.text_splitter import CharacterTextSplitter

# Sample Prolog-style KB facts and rules
kb_facts = """\
animal(dog).
animal(cat).
animal(eagle).
animal(sparrow).
flies(eagle).
flies(sparrow).
has_tail(dog).
has_tail(cat).
"""

kb_rules = """\
bird(X) :- animal(X), flies(X).
mammal(X) :- animal(X), has_tail(X).
"""

# Convert to LangChain Documents
docs = [
    Document(page_content=kb_facts, metadata={"source": "facts"}),
    Document(page_content=kb_rules, metadata={"source": "rules"})
]

# Split text into chunks
splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=0)
kb_chunks = splitter.split_documents(docs)

# Use HuggingFace embeddings (use default model: "all-MiniLM-L6-v2")
embedding = HuggingFaceEmbeddings()
vectorstore = FAISS.from_documents(kb_chunks, embedding)
retriever = vectorstore.as_retriever()

<ipython-input-15-792b2e82ad0d>:35: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedding = HuggingFaceEmbeddings()


In [ ]:
# sample test
retriever = vectorstore.as_retriever()
query = "Can a sparrow fly?"
retrieved_docs = retriever.invoke(query)

for doc in retrieved_docs:
    print("Source:", doc.metadata["source"])
    print("Content:", doc.page_content)
    print()


Source: facts
Content: animal(dog).
animal(cat).
animal(eagle).
animal(sparrow).
flies(eagle).
flies(sparrow).
has_tail(dog).
has_tail(cat).

Source: rules
Content: bird(X) :- animal(X), flies(X).
mammal(X) :- animal(X), has_tail(X).

